In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
from numpy import argmax
import pandas as pd
import tensorflow as tf
import seaborn as sns
import transformers
import pickle
import nltk
import re
import os, json
import keras
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import math
import torch
# import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, NLLLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
# from pytorch_transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from transformers import AdamW, XLNetTokenizer, XLNetModel, TFXLNetModel, XLNetLMHeadModel, XLNetConfig, XLNetForSequenceClassification
#from keras.utils.data_utils import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
import unicodedata
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score,precision_recall_curve
from sklearn.metrics import precision_recall_fscore_support
from numpy import trapz
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, roc_auc_score,balanced_accuracy_score
from sklearn.metrics import average_precision_score,cohen_kappa_score
from sklearn.model_selection import KFold
import seaborn as sns
import itertools
import plotly
import seaborn as sns
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter  
import time
import datetime
plt.style.use('seaborn')
#from transformers import TFXLNetModel, XLNetTokenizer
from transformers import XLNetTokenizer, XLNetModel
from nltk.tokenize import WordPunctTokenizer
import itertools
import statistics
from sklearn.utils import shuffle
import random

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+' 
 
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')    
def preprocess_sentence_english(w):

  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  
  w = re.sub(r"@[A-Za-z0-9]+", ' ', w)
  w = re.sub(r"https?://[A-Za-z0-9./]+", ' ', w)
  w = re.sub(r"[^a-zA-z.!?'0-9]", ' ', w)
  w = re.sub('\t', ' ',  w)
  w = re.sub(r" +", ' ', w)
    
 
  #chars (optional) - a string specifying the set of characters to be removed.
  #If the chars argument is not provided, all leading and trailing whitespaces are removed from the string.
  w = w.rstrip().strip()

 
  # Fix misspelled words
  w = ''.join(''.join(s)[:2] for _, s in itertools.groupby(w))# checking that each character should occur not more than 2 times in every word

  # Tokenizing ,change cases & join together to remove unneccessary white spaces
  w = tok.tokenize(w)
  w = (" ".join(w)).strip()
  return w

def tokenize_inputs(text_list, tokenizer, num_embeddings=120):
    """
    Tokenizes the input text input into ids. Appends the appropriate special
    characters to the end of the text to denote end of sentence. Truncate or pad
    the appropriate sequence length.
    """
    # tokenize the text, then truncate sequence to the desired length minus 2 for
    # the 2 special characters
    tokenized_texts = list(map(lambda t: tokenizer.tokenize(t)[:num_embeddings-2], text_list))
    # convert tokenized text into numeric ids for the appropriate LM
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    # append special token "<s>" and </s> to end of sentence
    input_ids = [tokenizer.build_inputs_with_special_tokens(x) for x in input_ids]
    # pad sequences
    input_ids = pad_sequences(input_ids, maxlen=num_embeddings, dtype="long", truncating="post", padding="post")
    return input_ids
    
    
def create_attn_masks(input_ids):
    """
    Create attention masks to tell model whether attention should be applied to
    the input id tokens. Do not want to perform attention on padding tokens.
    """
    # Create attention masks
    attention_masks = []

    # Create a mask of 1s for each token followed by 0s for padding
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    return attention_masks
    
    
class XLNetForMultiLabelSequenceClassification(torch.nn.Module):
  
  def __init__(self, num_labels=2):
    super(XLNetForMultiLabelSequenceClassification, self).__init__()
    self.num_labels = num_labels
    self.xlnet = XLNetModel.from_pretrained('xlnet-base-cased')
    self.classifier = torch.nn.Linear(768, num_labels)

    torch.nn.init.xavier_normal_(self.classifier.weight)

  def forward(self, input_ids, token_type_ids=None,\
              attention_mask=None, labels=None):
       
    # last hidden layer
    last_hidden_state = self.xlnet(input_ids=input_ids,\
                                   attention_mask=attention_mask,\
                                   token_type_ids=token_type_ids
                                  )
    # pool the outputs into a mean vector
    mean_last_hidden_state = self.pool_hidden_state(last_hidden_state)
    logits = self.classifier(mean_last_hidden_state)
   
    logits = logits[:, 1] - logits[:, 0]
    
    
    if labels is not None:

      loss = BCEWithLogitsLoss()(logits, labels.float())

      return loss
    else:
      return logits
    
  def freeze_xlnet_decoder(self):
    """
    Freeze XLNet weight parameters. They will not be updated during training.
    """
    for param in self.xlnet.parameters():
      param.requires_grad = False
    
  def unfreeze_xlnet_decoder(self):
    """
    Unfreeze XLNet weight parameters. They will be updated during training.
    """
    for param in self.xlnet.parameters():
      param.requires_grad = True
  def pool_hidden_state(self, last_hidden_state):
    """
    Pool the output vectors into a single mean vector 
    """
    last_hidden_state = last_hidden_state[0]
    mean_last_hidden_state = torch.mean(last_hidden_state, 1)
    return mean_last_hidden_state    
  

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


def train_def(model, num_epochs,\
          optimizer,\
          train_dataloader, valid_dataloader,\
          model_save_path,\
          train_loss_set=[], valid_loss_set = [],\
          lowest_eval_loss=None, start_epoch=0,\
          device="cpu"
          ):
  """
  Train the model and save the model with the lowest validation loss
  """
  # We'll store a number of quantities such as training and validation loss, 
  # validation accuracy, and timings.
  training_stats = []
  # Measure the total training time for the whole run.
  total_t0 = time.time()

  model.to(device)

  # trange is a tqdm wrapper around the python range function
  for i in trange(num_epochs, desc="Epoch"):
    # if continue training from saved model
    actual_epoch = start_epoch + i
   

    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set. 
    print("")
    print('======== Epoch {:} / {:} ========'.format(actual_epoch, num_epochs))
    print('Training...')
    
    # Measure how long the training epoch takes.
    t0 = time.time()
    
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Tracking variables
    tr_loss = 0
    num_train_samples = 0

    # Train the data for one epoch
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0: 
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        loss = model(input_ids=b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        # store train loss
        tr_loss += loss.item()
        num_train_samples += b_labels.size(0)
        # Backward pass
        loss.backward()
        # Gradient Clipping 
        torch.nn.utils.clip_grad_norm(parameters=model.parameters(), max_norm=max_grad_norm)
        # Update parameters and take a step using the computed gradient
        optimizer.step()
        #scheduler.step()

    # Update tracking variables
    epoch_train_loss = tr_loss/num_train_samples
    train_loss_set.append(epoch_train_loss)

#     print("Train loss: {}".format(epoch_train_loss))
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(epoch_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()
    
    # Put model in evaluation mode to evaluate loss on the validation set
    model.eval()

    # Tracking variables 
    eval_loss = 0
    num_eval_samples = 0

    # Evaluate data for one epoch
    for batch in valid_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients,
        # saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate validation loss
            loss = model(input_ids=b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            # store valid loss
            eval_loss += loss.item()
            num_eval_samples += b_labels.size(0)

    epoch_eval_loss = eval_loss/num_eval_samples
    valid_loss_set.append(epoch_eval_loss)

#     print("Valid loss: {}".format(epoch_eval_loss))
    
    # Report the final accuracy for this validation run.
#     avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
#     print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
#     avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(epoch_eval_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': actual_epoch,
            'Training Loss': epoch_train_loss,
            'Valid. Loss': epoch_eval_loss,
#             'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

    
    if lowest_eval_loss == None:
      lowest_eval_loss = epoch_eval_loss
      # save model
      save_model(model, model_save_path, actual_epoch,\
                 lowest_eval_loss, train_loss_set, valid_loss_set)
    else:
      if epoch_eval_loss < lowest_eval_loss:
        lowest_eval_loss = epoch_eval_loss
        # save model
        save_model(model, model_save_path, actual_epoch,\
                   lowest_eval_loss, train_loss_set, valid_loss_set)
  
  print("")
  print("Training complete!")

  print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
  return model, train_loss_set, valid_loss_set, training_stats  



# function to save and load the model form a specific epoch
def save_model(model, save_path, epochs, lowest_eval_loss, train_loss_hist, valid_loss_hist):
  """
  Save the model to the path directory provided
  """
  model_to_save = model.module if hasattr(model, 'module') else model
  checkpoint = {'epochs': epochs, \
                'lowest_eval_loss': lowest_eval_loss,\
                'state_dict': model_to_save.state_dict(),\
                'train_loss_hist': train_loss_hist,\
                'valid_loss_hist': valid_loss_hist,\
                'optimizer_state_dict': optimizer.state_dict()
               }
               
  torch.save(checkpoint, save_path)
  print("Saving model at epoch {} with validation loss of {}".format(epochs,\
                                                                     lowest_eval_loss))
  return
def load_model(save_path):
  """
  Load the model from the path directory provided
  """
  checkpoint = torch.load(save_path)
  model_state_dict = checkpoint['state_dict']
  model = XLNetForMultiLabelSequenceClassification(num_labels=model_state_dict["classifier.weight"].size()[0])
  model.load_state_dict(model_state_dict)

  epochs = checkpoint["epochs"]
  lowest_eval_loss = checkpoint["lowest_eval_loss"]
  train_loss_hist = checkpoint["train_loss_hist"]
  valid_loss_hist = checkpoint["valid_loss_hist"]
  
  return model, epochs, lowest_eval_loss, train_loss_hist, valid_loss_hist 

def generate_predictions(model, df, device="cpu", batch_size=16):
  num_iter = math.ceil(df.shape[0]/batch_size)
  
  pred_probs = []

  model.to(device)
  model.eval()
  
  for i in range(num_iter):
    df_subset = df.iloc[i*batch_size:(i+1)*batch_size,:]
    X = df_subset["features"].values.tolist()
    masks = df_subset["masks"].values.tolist()
    X = torch.tensor(X)
    masks = torch.tensor(masks, dtype=torch.long)
    X = X.to(device)
    masks = masks.to(device)
    with torch.no_grad():
      logits = model(input_ids=X, attention_mask=masks)
      logits = logits.sigmoid().detach().cpu().numpy()
#       pred_probs = np.vstack([pred_probs, logits])
      pred_probs.extend(logits.tolist())
        
  return pred_probs


def Evaluate(labels, predictions):
    avg = sum(predictions)/len(predictions)
    p=avg
    CM= confusion_matrix(labels, predictions > p)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    print(' (True Negatives): {}'.format(TN))
    print(' (False Negatives):  {}'.format(FN))
    print(' (True Positives): {}'.format(TP))
    print('(False Positives):{}'.format(FP))
    print('Total positive : ', np.sum(CM[1]))
    auc = roc_auc_score(labels, predictions)
    prec=precision_score(labels, predictions>p)
    rec=recall_score(labels, predictions>p)
     # calculate F1 score
    f1 = f1_score(labels, predictions>p)
    print('auc :{}'.format(auc))
    print('precision :{}'.format(prec))
    print('recall :{}'.format(rec))
    print('f1 :{}'.format(f1))
    # Compute Precision-Recall and plot curve
    precision, recall, thresholds = precision_recall_curve(labels, predictions >p)
    #use the trapezoidal rule to calculate the area under the precion-recall curve
    area =  trapz(recall, precision)
   
    #area =  simps(recall, precision)
    print("Area Under Precision Recall  Curve(AP): %0.4f" % area)   #should be same as AP? 
    average_precision = average_precision_score(labels, predictions>p)
    print("average precision: %0.4f" % average_precision)
    kappa = cohen_kappa_score(labels, predictions>p)
    print('kappa :{}'.format(kappa))
    balanced_accuracy = balanced_accuracy_score(labels, predictions>p)
    print('balanced_accuracy :{}'.format(balanced_accuracy))
    return prec, rec, f1
    
def freeze(model, depth):
    if depth:
        model.xlnet.word_embedding.requires_grad_(False)
        for d in range(depth):
              model.xlnet.layer[d].rel_attn.layer_norm.requires_grad_(False)
              model.xlnet.layer[d].rel_attn.dropout.p = 0.
              model.xlnet.layer[d].ff.layer_norm.requires_grad_(False)
              model.xlnet.layer[d].ff.layer_1.requires_grad_(False)
              model.xlnet.layer[d].ff.layer_2.requires_grad_(False)
              model.xlnet.layer[d].ff.dropout.p = 0.
              model.xlnet.layer[d].dropout.p = 0.

def freeze_params(model):
        for param in model.parameters():
            param.requires_grad = False


if __name__ == '__main__':
    k_folds = 10
    max_grad_norm = 1.0
    batch_size = 16
    num_epochs = 10
    MAX_LEN = 250
    #Data Augmentation
    train = pd.read_csv(r'../input/stress-analysis-in-social-media/dreaddit-train.csv')
    train_x= train.text.tolist()
    train_y= train.label.tolist()
    X = train_x 
    y = train_y 
    data = list(zip(X, y))
    random.shuffle(data)
    data = pd.DataFrame(data, columns=['text','tag'])
    X = data.text.tolist()
    y = data.tag.tolist()
    X = np.array(X)
    y = np.array(y)
    # Set fixed random number seed
    torch.manual_seed(42)
     # Define the K-fold Cross Validator
    kfold = KFold(n_splits=k_folds, shuffle=True)
    P = []
    R = []
    F1 = []
    for train_index, test_index in kfold.split(X, y):
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]
      data_train = list(zip(X_train,y_train))
      data_test = list(zip(X_test,y_test))
      train = pd.DataFrame(data_train, columns=['text','tag'])
      test = pd.DataFrame(data_test, columns=['text','tag'])
      print('Number of training sentences: {:,}\n'.format(train.shape[0]))
      print('Number of testing sentences: {:,}\n'.format(test.shape[0]))
      
            
      tokenizer = XLNetTokenizer.from_pretrained('xlnet-large-cased')
      model = XLNetLMHeadModel.from_pretrained('xlnet-large-cased')
      
      # cleaning tweets
      train['text_cleaned'] = list(map(lambda x: preprocess_sentence_english(x),train['text']) )
      test['text_cleaned'] = list(map(lambda x: preprocess_sentence_english(x),test['text']) )
      
      sentences = train.text_cleaned.values
      labels = train.tag.values
            
      # Tokenize all of the sentences and map the tokens to thier word IDs.
      
      input_ids = tokenize_inputs(sentences, tokenizer, num_embeddings=MAX_LEN)
      attention_masks = create_attn_masks(input_ids)
      input_ids = torch.from_numpy(input_ids)
      attention_masks = torch.tensor(attention_masks)
      labels = torch.tensor(labels)
    
      test_sentences = test.text_cleaned.values
      test_labels = test.tag.values
            
      # Tokenize all of the sentences and map the tokens to thier word IDs.
      
      test_input_ids = tokenize_inputs(test_sentences, tokenizer, num_embeddings=MAX_LEN)
      test_attention_masks = create_attn_masks(test_input_ids)
      test_input_ids = torch.from_numpy(test_input_ids)
      test_attention_masks = torch.tensor(test_attention_masks)
      test_labels = torch.tensor(test_labels)

     

    
    
    
      
      # Combine the training inputs into a TensorDataset.
      train_dataset = TensorDataset(input_ids, attention_masks, labels)
      
      val_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)
      
      # Divide the dataset by randomly selecting samples.
      #train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
      
      # Checking whether the distribution of target is consitent across both the sets
      label_temp_list = []
      for a,b,c in train_dataset:
        label_temp_list.append(c)
      
     
      print('{:>5,} training samples with stress'.format(sum(label_temp_list)))
      
      label_temp_list = []
      for a,b,c in val_dataset:
        label_temp_list.append(c)
      
      
      print('{:>5,} validation samples with stress'.format(sum(label_temp_list)))
      
      
      # Create the DataLoaders for our training and validation sets.
      # We'll take training samples in random order. 
      train_dataloader = DataLoader(
                  train_dataset,  # The training samples.
                  sampler = RandomSampler(train_dataset), # Select batches randomly
                  batch_size = batch_size # Trains with this batch size.
              )
      
      # For validation the order doesn't matter, so we'll just read them sequentially.
      validation_dataloader = DataLoader(
                  val_dataset, # The validation samples.
                  sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
                  batch_size = batch_size # Evaluate with this batch size.
              )
      model = XLNetForMultiLabelSequenceClassification(num_labels=len(labels.unique()))
      param_optimizer = list(model.named_parameters())
      no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
      optimizer_grouped_parameters = [
                                      {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                      {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}
      ]
      optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)
      
      torch.cuda.empty_cache()
      num_epochs = 10
      
      cwd = os.getcwd()
      
      model_save_path = output_model_file = "/xlnet.bin" 
      
      
      model, train_loss_set, valid_loss_set, training_stats = train_def(model=model,\
                                                                    num_epochs=num_epochs,\
                                                                    optimizer=optimizer,\
                                                                    train_dataloader=train_dataloader,\
                                                                    valid_dataloader=validation_dataloader,\
                                                                    model_save_path=model_save_path,\
                                                                    device="cuda"#"cuda"
                                                                    )
      
    save_path = "/xlnet.pth"
      

    torch.save(model.state_dict(), save_path)

    device = torch.device("cuda")
      #model = TheModelClass(*args, **kwargs)
    model = XLNetForMultiLabelSequenceClassification(num_labels=len(labels.unique()))
    model.load_state_dict(torch.load(save_path))
    model.to(device)
    test = pd.read_csv(r'../input/stress-analysis-in-social-media/dreaddit-test.csv')
    test_x = test.text.tolist()
    test_y = test.label.tolist()
    data = list(zip(test_x, test_y))
    test = pd.DataFrame(data_test, columns=['text','tag'])

    test['text_cleaned'] = list(map(lambda x:  preprocess_sentence_english(x),test['text']) )

    # Get the lists of sentences and their labels
    sentences = test.text_cleaned.values

      # input_ids = torch.from_numpy(input_ids)
      # attention_masks = torch.tensor(attention_masks)
      # labels = torch.tensor(labels)

    test_input_ids = tokenize_inputs(sentences, tokenizer, num_embeddings=MAX_LEN)
    test_attention_masks = create_attn_masks(test_input_ids)

    test["features"] = test_input_ids.tolist()
    test["masks"] = test_attention_masks
    pred_probs = generate_predictions(model, test, device="cuda", batch_size=4)
      # pred_probs

    statistics.mean(pred_probs)
    test['target'] = pred_probs
    test['target'] = np.array(test['target'] >= 0.5, dtype='int')

    auc_value = roc_auc_score(test['tag'], np.asarray(pred_probs) >0.5)
    print("auc  on test {}".format(auc_value))
    prec, rec, f1 = Evaluate(test['tag'], np.asarray(pred_probs) >0.5) 
    print(prec)
    print(rec)
    print(f1)
     

In [1]:
# reference - http://restanalytics.com/2021-05-04-Fine-Tuning-XLNet-For-Sequence-Classification/